In [34]:
import pandas as pd
import numpy as np
import os
import re
import pickle
from bs4 import BeautifulSoup as soup
import requests as rq
import time
import h5py
import scipy as sp
import scipy.stats

# EDA

## Pulling in Data

The data collected is a mix of HDF5, csv and web scraped data about the Bay area.  The CSVs are for particle concentraions, HDF5s for CO2 ppm, and the web scraped data is for empty lot/apartments for sale in the San Fransisco.

In [323]:
#starting off by reading in the csv files
aqcsvs = []
aqcsvs += ['../../../../Documents/largecsvs/others/'+each for each in os.listdir('../../../../Documents/largecsvs/others/') if each.endswith('.csv')]

part = []
part += ['../../../../Documents/largecsvs/particle/'+each for each in os.listdir('../../../../Documents/largecsvs/particle/') if each.endswith('csv')]
#getting all H5 files for the project
results = []
results += ['../../../../Downloads/'+each for each in os.listdir('../../../../Downloads/') if each.endswith('.h5')]

In [324]:
aqcsvs=sorted(aqcsvs)
part=sorted(part)

In [325]:
weatherdf = pd.read_csv('weather/1367384.csv')

In [326]:
weatherdf.drop(['STATION', 'NAME','ELEVATION', 'SNWD', 'TOBS'], axis=1, inplace=True)
weatherdf = weatherdf.fillna(0)

In [327]:
weatherdf.head()

,LATITUDE,LONGITUDE,DATE,PRCP,SNOW,TAVG,TMAX,TMIN
0,37.7705,-122.4269,2014-01-01,0.0,0.0,0.0,63.0,44.0
1,37.7705,-122.4269,2014-01-02,0.0,0.0,0.0,63.0,48.0
2,37.7705,-122.4269,2014-01-03,0.0,0.0,0.0,64.0,49.0
3,37.7705,-122.4269,2014-01-04,0.0,0.0,0.0,66.0,46.0
4,37.7705,-122.4269,2014-01-05,0.0,0.0,0.0,66.0,48.0


In [328]:
part[]

['../../../../Documents/largecsvs/particle/Pdaily_88101_2014.csv',
 '../../../../Documents/largecsvs/particle/Pdaily_88101_2015.csv',
 '../../../../Documents/largecsvs/particle/Pdaily_88101_2016.csv',
 '../../../../Documents/largecsvs/particle/Pdaily_88101_2017.csv',
 '../../../../Documents/largecsvs/particle/Pdaily_88101_2018.csv']

In [486]:
pdf=pd.read_csv(part[0])

for i in range(1,len(part)):
    temp = pd.read_csv(part[i])
    pdf = pd.concat([pdf,temp], ignore_index=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [487]:
#Filtering out unneed data
pdf = pdf[pdf['State Name']=='California']
pdf = pdf[pdf['CBSA Name']=='San Francisco-Oakland-Hayward, CA']
pdf = pdf[pdf['City Name'].isin(['Oakland','Richmond','San Pablo','San Francisco','San Rafael'])]
pdf.drop(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Datum', 'Sample Duration',
       'Pollutant Standard', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name','City Name',
       'CBSA Name', 'Date of Last Change'], axis=1, inplace=True)

pdf.reset_index(drop=True,inplace=True)

In [489]:
len(pdf)

17711

In [490]:
len(codf)

18132

In [491]:
aqcsvs

['../../../../Documents/largecsvs/others/COdaily_42101_2014.csv',
 '../../../../Documents/largecsvs/others/COdaily_42101_2015.csv',
 '../../../../Documents/largecsvs/others/COdaily_42101_2016.csv',
 '../../../../Documents/largecsvs/others/COdaily_42101_2017.csv',
 '../../../../Documents/largecsvs/others/COdaily_42101_2018.csv',
 '../../../../Documents/largecsvs/others/Odaily_44201_2014.csv',
 '../../../../Documents/largecsvs/others/Odaily_44201_2015.csv',
 '../../../../Documents/largecsvs/others/Odaily_44201_2016.csv',
 '../../../../Documents/largecsvs/others/Odaily_44201_2017.csv',
 '../../../../Documents/largecsvs/others/Odaily_44201_2018.csv',
 '../../../../Documents/largecsvs/others/SOdaily_42401_2014.csv',
 '../../../../Documents/largecsvs/others/SOdaily_42401_2015.csv',
 '../../../../Documents/largecsvs/others/SOdaily_42401_2016.csv',
 '../../../../Documents/largecsvs/others/SOdaily_42401_2017.csv',
 '../../../../Documents/largecsvs/others/SOdaily_42401_2018.csv',
 '../../../../D

In [492]:
#Creating a 3 dataframes based on which ones I will need, 2 are needed from this call
aq=pd.read_csv(aqcsvs[0])

for i in range(1,len(aqcsvs)):
    if i==15:
        aq2=pd.read_csv(aqcsvs[i])
    elif i>15 and i<=20:
        temp = pd.read_csv(aqcsvs[i])
        aq2 = pd.concat([aq2,temp], ignore_index=True)
    elif i==21:
        aq3=pd.read_csv(aqcsvs[i])
    elif i>21:
        temp = pd.read_csv(aqcsvs[i])
        aq3 = pd.concat([aq3,temp], ignore_index=True)
    else:
        temp = pd.read_csv(aqcsvs[i])
        aq = pd.concat([aq,temp], ignore_index=True)

In [493]:
#Filtering out unneed data
aq = aq[aq['State Name']=='California']
aq = aq[aq['CBSA Name']=='San Francisco-Oakland-Hayward, CA']
aq = aq[aq['City Name'].isin(['Oakland','Richmond','San Pablo','San Francisco','San Rafael'])]
aq.drop(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Datum', 'Sample Duration',
       'Pollutant Standard', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name','City Name',
       'CBSA Name', 'Date of Last Change'], axis=1, inplace=True)
#filtering to relevant cities
aq.reset_index(drop=True,inplace=True)

In [494]:
codf = aq[aq['Parameter Name']=='Carbon monoxide']
ozdf= aq[aq['Parameter Name']=='Ozone']
sodf= aq[aq['Parameter Name']=='Sulfur dioxide']


In [495]:
aq3 = aq3[['county Name', 'Date', 'AQI','Category', 'Defining Parameter']]
aq3.columns

Index(['county Name', 'Date', 'AQI', 'Category', 'Defining Parameter'], dtype='object')

In [496]:
#Filtering out unneed data
aq3 = aq3[aq3['county Name'].isin(aq['County Name'].unique())]
#filtering to relevant cities
aq3.reset_index(drop=True,inplace=True)
aq3.head()

,county Name,Date,AQI,Category,Defining Parameter
0,Alameda,2015-01-01,61,Moderate,PM2.5
1,Alameda,2015-01-02,76,Moderate,PM2.5
2,Alameda,2015-01-03,96,Moderate,PM2.5
3,Alameda,2015-01-04,99,Moderate,PM2.5
4,Alameda,2015-01-05,91,Moderate,PM2.5


In [507]:
codf.columns=['Latitude', 'Longitude', 'Param Name', 'Date Local',
       'CO Mean', 'CO Max Value', 'CO Max Hour','County Name']
ozdf.columns=['Latitude', 'Longitude', 'Param Name2', 'Date Local',
       'Oz Mean', 'Oz Max Value', 'Oz Max Hour','County Name']
sodf.columns=['Latitude', 'Longitude', 'Param Name3', 'Date Local',
       'SO Mean', 'SO Max Value', 'SO Max Hour','County Name']
aq3.columns=['County Name', 'Date Local', 'AQI', 'Category', 'AQ Defining Parameter']
pdf.columns =['Latitude', 'Longitude', 'Param Name4', 'Date Local','Part Mean', 'Part Max Value', 'Part Max Hour','County Name']


In [498]:
aq = pd.merge(codf,ozdf, on=['Latitude', 'Longitude', 'Date Local','County Name'])
aq2 = pd.merge(aq,sodf, on=['Latitude', 'Longitude', 'Date Local','County Name'])


In [500]:
qq = pd.merge(aq2,qq, how='inner', on=['Latitude', 'Longitude', 'Date Local','County Name'])


In [508]:
aq = pd.merge(qq,aq3, on=['Date Local','County Name'])

In [509]:
aq.sample(10)

,Latitude,Longitude,Param Name,Date Local,CO Mean,CO Max Value,CO Max Hour,County Name,Param Name2,Oz Mean,...,SO Mean,SO Max Value,SO Max Hour,Param Name4,Part Mean,Part Max Value,Part Max Hour,AQI,Category,AQ Defining Parameter
12817,37.814781,-122.282347,Carbon monoxide,2017-09-24,0.565217,1.1,2,Alameda,Ozone,0.017824,...,0.163636,0.7,11,PM2.5 - Local Conditions,14.100000,14.1,0,55,Moderate,PM2.5
5653,37.814781,-122.282347,Carbon monoxide,2016-01-25,0.678261,1.1,7,Alameda,Ozone,0.008471,...,0.868182,1.9,13,PM2.5 - Local Conditions,8.200000,8.2,0,41,Good,PM2.5
4741,37.960400,-122.356811,Carbon monoxide,2015-09-18,0.465217,1.0,6,Contra Costa,Ozone,0.018235,...,0.700000,1.5,11,PM2.5 - Local Conditions,6.437500,12.0,9,45,Good,Ozone
15776,37.960400,-122.356811,Carbon monoxide,2017-10-05,0.495833,0.6,7,Contra Costa,Ozone,0.019529,...,1.026087,3.0,12,PM2.5 - Local Conditions,7.958333,13.0,18,42,Good,Ozone
11540,37.814781,-122.282347,Carbon monoxide,2017-04-05,0.478261,0.7,4,Alameda,Ozone,0.040765,...,0.904545,2.4,11,PM2.5 - Local Conditions,9.958333,17.0,11,48,Good,Ozone
3443,37.960400,-122.356811,Carbon monoxide,2015-03-27,0.329167,0.4,0,Contra Costa,Ozone,0.019176,...,0.742857,0.9,8,PM2.5 - Local Conditions,2.500000,2.5,0,31,Good,Ozone
12425,37.814781,-122.282347,Carbon monoxide,2017-08-02,0.375000,0.4,6,Alameda,Ozone,0.016059,...,0.827273,1.1,16,PM2.5 - Local Conditions,6.300000,6.3,0,97,Moderate,Ozone
8339,37.960400,-122.356811,Carbon monoxide,2016-02-04,0.486957,0.7,21,Contra Costa,Ozone,0.018471,...,0.885714,1.1,11,PM2.5 - Local Conditions,10.300000,10.3,0,43,Good,PM2.5
2214,37.814781,-122.282347,Carbon monoxide,2015-10-20,0.828571,1.7,11,Alameda,Ozone,0.009176,...,1.014286,2.4,8,PM2.5 - Local Conditions,7.782609,14.0,7,36,Good,Ozone
13647,37.960400,-122.356811,Carbon monoxide,2017-01-09,0.408333,0.5,8,Contra Costa,Ozone,0.034588,...,0.900000,1.1,2,PM2.5 - Local Conditions,5.600000,5.6,0,35,Good,Ozone


In [510]:
aq.drop(['Param Name', 'County Name', 'Param Name2','Param Name3','Param Name4'], axis=1)

,Latitude,Longitude,Date Local,CO Mean,CO Max Value,CO Max Hour,Oz Mean,Oz Max Value,Oz Max Hour,SO Mean,SO Max Value,SO Max Hour,Part Mean,Part Max Value,Part Max Hour,AQI,Category,AQ Defining Parameter
0,37.814781,-122.282347,2015-01-01,0.708696,2.2,21,0.015882,0.034,9,1.050000,1.5,16,13.000000,35.0,21,61,Moderate,PM2.5
1,37.814781,-122.282347,2015-01-01,0.708696,2.2,21,0.015882,0.034,9,1.050000,1.5,16,13.000000,13.0,0,61,Moderate,PM2.5
2,37.814781,-122.282347,2015-01-01,0.708696,2.2,21,0.015882,0.034,9,1.042857,1.4,23,13.000000,35.0,21,61,Moderate,PM2.5
3,37.814781,-122.282347,2015-01-01,0.708696,2.2,21,0.015882,0.034,9,1.042857,1.4,23,13.000000,13.0,0,61,Moderate,PM2.5
4,37.814781,-122.282347,2015-01-01,0.533333,1.3,23,0.015882,0.034,9,1.050000,1.5,16,13.000000,35.0,21,61,Moderate,PM2.5
5,37.814781,-122.282347,2015-01-01,0.533333,1.3,23,0.015882,0.034,9,1.050000,1.5,16,13.000000,13.0,0,61,Moderate,PM2.5
6,37.814781,-122.282347,2015-01-01,0.533333,1.3,23,0.015882,0.034,9,1.042857,1.4,23,13.000000,35.0,21,61,Moderate,PM2.5
7,37.814781,-122.282347,2015-01-01,0.533333,1.3,23,0.015882,0.034,9,1.042857,1.4,23,13.000000,13.0,0,61,Moderate,PM2.5
8,37.814781,-122.282347,2015-01-02,1.056522,2.4,23,0.011471,0.021,9,1.859091,5.6,12,24.041667,44.0,23,76,Moderate,PM2.5
9,37.814781,-122.282347,2015-01-02,1.056522,2.4,23,0.011471,0.021,9,1.859091,5.6,12,24.000000,24.0,0,76,Moderate,PM2.5


In [22]:
def h5ToDF(file):
    f = h5py.File(file, 'r')
    date = [pd.to_datetime(str(x).replace('b\'','').replace('\'','')) for x in list(f['RetrievalHeader']['retrieval_time_string'])]
    latlong = [(x,y) for x,y in zip(f['RetrievalGeometry']['retrieval_latitude_geoid'],f['RetrievalGeometry']['retrieval_longitude_geoid'])]
    co2ppm = list(f['RetrievalResults']['xco2'])
    co2_uncert = [x*10**6 for x in list(f['RetrievalResults']['xco2_uncert'])]
    df = pd.DataFrame({'date':date,
        'latlong': latlong,
        'co2ppm': co2ppm,
        'co2_uncert': co2_uncert
    })
    return df
                                                                                    
                                                                                    
                                                                                    

In [23]:
df = f = h5py.File(results[0], 'r')
date = [pd.to_datetime(str(x).replace('b\'','').replace('\'','')) for x in list(f['RetrievalHeader']['retrieval_time_string'])]
latlong = [(x,y) for x,y in zip(f['RetrievalGeometry']['retrieval_latitude_geoid'],f['RetrievalGeometry']['retrieval_longitude_geoid'])]
co2ppm = list(f['RetrievalResults']['xco2'])
co2_uncert = [x*10**6 for x in list(f['RetrievalResults']['xco2_uncert'])]
df = pd.DataFrame({'date':date,
                'latlong': latlong,
                'co2ppm': co2ppm,
                'co2_uncert': co2_uncert})
for i in range(1,len(results)):
    df.append(h5ToDF(results[i]))

In [24]:
df['co2ppm'] = [x*10**6 for x in df['co2ppm']]

In [28]:
df['co2_uncert'] = [float(x) for x in df['co2_uncert']]

In [29]:
df.head()

,co2_uncert,co2ppm,date,latlong
0,0.886318,402.186939,2016-11-13 20:26:38.363,"(-83.56647, 38.90449)"
1,0.786782,398.468226,2016-11-13 20:26:38.390,"(-83.59027, 39.013676)"
2,0.780767,396.430114,2016-11-13 20:26:38.417,"(-83.61452, 39.12222)"
3,0.777430,398.074626,2016-11-13 20:26:38.562,"(-83.46712, 38.238407)"
4,0.799400,399.525161,2016-11-13 20:26:38.616,"(-83.51127, 38.46135)"


In [30]:
#Writing a pickle file to save data
import pickle
with open('my_data.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [ ]:
#list(f['RetrievalGeometry']['retrieval_latitude_geoid'])

## Web Scraping

Pulling data from loopnet.com for areas where we can greenify citys (make them more environmentally friendly). 

In [97]:
#initializing sites that I will pull from
flexspace = 'http://www.loopnet.com/california/san-francisco_flex-space-for-sale/'
land = 'http://www.loopnet.com/california/san-francisco_land-for-sale/'
apts = 'http://www.loopnet.com/california/san-francisco_apartment-buildings-for-sale/'

In [98]:
response = rq.get(land)
page = response.text
reviewp = soup(page,"lxml")


In [99]:
reviewp.find_all(class_='listingTitle')

[<span class="listingTitle">0.05 Acre Site for Proposed Apartments</span>,
 <span class="listingTitle">1228 Folsom Street</span>,
 <span class="listingTitle">1876 Oakdale Ave</span>,
 <span class="listingTitle">1791 Mission Street</span>,
 <span class="listingTitle">240 7th St</span>,
 <span class="listingTitle">1924 Mission St</span>,
 <span class="listingTitle">2435-2445 16th St</span>,
 <span class="listingTitle">Topaz Way</span>,
 <span class="listingTitle">Portfolio of 4 Properties</span>,
 <span class="listingTitle">1234 Shoreline Hwy</span>,
 <span class="listingTitle">81 Ervine St</span>,
 <span class="listingTitle">OCTAVIA M+N</span>,
 <span class="listingTitle">250 Church St</span>,
 <span class="listingTitle">Mission and 22nd</span>]

## Exploring

Exploring the data, kicking out points that are wrong based on the error and other stats.

In [33]:
print('Max: '+ str(max(df['co2_uncert'])))
print('Min: ' + str(min(df['co2_uncert'])))
print('Average: '+ str(np.average(df['co2_uncert'])))
print('Median: '+ str(np.median(df['co2_uncert'])))

Max: 13.534934623748995
Min: 0.1912240037427182
Average: 0.4512936642967768
Median: 0.4234372852351953


In [46]:
np.percentile(df['co2_uncert'], 99.5)

1.041737527884833

In [51]:
#Removing Outliers because of large error associated with the calculation
df= (df[df['co2_uncert']<1.04])